# HXRSnD Run

In [1]:
# Jupyter Notebook Settings
# This makes it so functions are automatically reloaded everytime they are 
# called, that way it isn't necessary to rerun the whole notebook after
# every change

# Load the "autoreload" extension
%load_ext autoreload
# always reload modules marked with "%aimport". 'autoreload 2' will set 
# everything to be autoreloaded except for what has %aimport
%autoreload 1
# Load matplotlib plots in interactive notebook mode
%matplotlib notebook

In [2]:
# Handle all the imports

############
# Standard #
############
import logging
from imp import reload

###############
# Third Party #
###############
from bluesky import RunEngine
from bluesky.plans import run_wrapper

########
# SLAC #
########
%aimport pcdsdevices.daq
from pcdsdevices.daq import make_daq_run_engine

##########
# Module #
##########
%aimport hxrsnd.plans
from hxrsnd.plans import linear_scan
%aimport hxrsnd.utils
from hxrsnd.utils import get_logger
%aimport hxrsnd.pneumatic
from hxrsnd.pneumatic import SndPneumatics
%aimport hxrsnd.sndsystem
from hxrsnd.sndsystem import SplitAndDelay
%aimport hxrsnd.tower
from hxrsnd.tower import DelayTower, ChannelCutTower
%aimport hxrsnd.bragg
from hxrsnd.bragg import bragg_angle, bragg_energy, sind, cosd
%aimport hxrsnd.diode
from hxrsnd.diode import HamamatsuXMotionDiode, HamamatsuXYMotionCamDiode

In [3]:
# Logging
logger = get_logger(__name__)
get_logger("pcdsdevices")

<Logger pcdsdevices (DEBUG)>

In [4]:
# Instantiate the system
pv_base = "XCS:SND"

# The whole system
snd = SplitAndDelay(pv_base)

# Towers
t1 = DelayTower(pv_base + ":T1", y1="A:ACT0", y2="A:ACT1", chi1="A:ACT2",
                chi2="B:ACT0", dh="B:ACT1", pos_inserted=21.1,
                pos_removed=0, desc="Tower 1")
t2 = ChannelCutTower(pv_base + ":T2", pos_inserted=None, pos_removed=0, 
                     desc="Tower 2")
t3 = ChannelCutTower(pv_base + ":T3", pos_inserted=None, pos_removed=0, 
                     desc="Tower 3")
t4 = DelayTower(pv_base + ":T4", y1="C:ACT0", y2="C:ACT1", chi1="C:ACT2",
                chi2="D:ACT0", dh="D:ACT1", pos_inserted=21.1,
                pos_removed=0, desc="Tower 4")

# Vacuum
ab = SndPneumatics(pv_base)

# Diagnostics
di = HamamatsuXMotionDiode(pv_base + ":DIA:DI")
dd = HamamatsuXYMotionCamDiode(pv_base + ":DIA:DD")
do = HamamatsuXMotionDiode(pv_base + ":DIA:DO")
dci = HamamatsuXMotionDiode(pv_base + ":DIA:DCI")
dcc = HamamatsuXYMotionCamDiode(pv_base + ":DIA:DCC")
dco = HamamatsuXMotionDiode(pv_base + ":DIA:DCO")

In [6]:
snd

Split and Delay System Status
-----------------------------
Delay Energy:          9000      
CC Energy:             9500      
Delay:           58.49625977004352

Tower 1
  --------------------------------------------------
  Motor           |    Position    |      Dial      
  --------------------------------------------------
  T1 TTH          |     42.043     |     -3.749     
  T1 TH1          |     21.022     |    -141.763    
  T1 TH2          |     21.022     |    -98.220     
  T1 X            |     0.000      |    -20.610     
  T1 L            |    179.323     |     91.158     
  --------------------------------------------------
  Motor           |    Position    |   Reference    
  --------------------------------------------------
  T1 Y1           |     -0.580     |     0.000      
  T1 Y2Y          |     -2.500     |     0.000      
  T1 CHI1         |     -0.100     |     0.000      
  T1 CHI2         |     0.250      |     0.000      
  T1 DH           |     -2.000   